In [1]:
from pymongo import MongoClient
import pandas as pd
import pymongo
import folium
from folium import plugins
from folium.plugins import MiniMap
from folium.plugins import FastMarkerCluster
import numpy as np
import re
import requests as req

In [2]:
#Connecting the database with the queried data (geo_offices)

client = MongoClient('mongodb://localhost:27017/')
db = client.DBcompanies_cb
data = db.companies_clean.find()

In [3]:
#Creating the dataframe

df_comp = pd.DataFrame(data)
df_comp.head()

,_id,amount_raised_k$,category_code,geopoint,lat,lng,name,number_of_employees,wealth
0,5d62c53b087b7af24fa9918e,45000.0,news,"{'type': 'Point', 'coordinates': [-122.394523,...",37.764726,-122.394523,Digg,60,0.172518
1,5d62c53b087b7af24fa9918f,39800.0,web,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47,0.133591
2,5d62c53b087b7af24fa99190,39800.0,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",40.723731,-73.996431,Wetpaint,47,0.133591
3,5d62c53b087b7af24fa99191,16500.0,web,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18,0.046909
4,5d62c53b087b7af24fa99192,25800.0,news,"{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052,Scribd,50,0.136301


In [4]:
print(df_comp.shape)
print(df_comp.index)

(2471, 9)
RangeIndex(start=0, stop=2471, step=1)


In [5]:
#Ordering the columns and dropping column id.

data_comp = df_comp[['name', 'category_code', 'number_of_employees', 'amount_raised_k$', 'wealth', 'lat', 'lng', 'geopoint']]
data_comp.head()


,name,category_code,number_of_employees,amount_raised_k$,wealth,lat,lng,geopoint
0,Digg,news,60,45000.0,0.172518,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,Wetpaint,web,47,39800.0,0.133591,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
2,Wetpaint,web,47,39800.0,0.133591,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,Geni,web,18,16500.0,0.046909,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
4,Scribd,news,50,25800.0,0.136301,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,..."


In [6]:
#Querying near companies

def nearComps(df, rad_max_meters=1000):
    lst=[]
    for i in range(len(df)): 
        near=db.companies_clean.find({'$and':[{
                        "geopoint": {
                            "$near": {
                            "$geometry": df[i],
                            "$maxDistance": rad_max_meters
                            }
                        }
                    }]})
        data=pd.DataFrame(near)
        lst.append(data.shape[0])
    return lst

data_comp['offices_near']=nearComps(data_comp["geopoint"])

In [9]:
data_comp.head()

,name,category_code,number_of_employees,amount_raised_k$,wealth,lat,lng,geopoint,offices_near
0,Digg,news,60,45000.0,0.172518,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",6
1,Wetpaint,web,47,39800.0,0.133591,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",15
2,Wetpaint,web,47,39800.0,0.133591,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,...",25
3,Geni,web,18,16500.0,0.046909,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1
4,Scribd,news,50,25800.0,0.136301,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,...",67


In [10]:
data_comp.sort_values('offices_near', ascending = False).head()

,name,category_code,number_of_employees,amount_raised_k$,wealth,lat,lng,geopoint,offices_near
1629,Green Apple Media,web,50,132000.0,0.158205,37.785271,-122.397582,"{'type': 'Point', 'coordinates': [-122.397582,...",93
407,GotVoice,mobile,12,3000.0,0.025783,37.786464,-122.397819,"{'type': 'Point', 'coordinates': [-122.397819,...",90
198,TrustedID,security,120,25000.0,0.326108,37.786240,-122.398970,"{'type': 'Point', 'coordinates': [-122.39897, ...",89
166,Veodia,games_video,15,8300.0,0.036325,37.786906,-122.397672,"{'type': 'Point', 'coordinates': [-122.397672,...",88
305,Covario,enterprise,100,16000.0,0.259780,37.787183,-122.397759,"{'type': 'Point', 'coordinates': [-122.397759,...",86


In [11]:
#Making business with a very large companies is not very easy if your startup is big enough. I cannot see SpaceX buying a bunch of nanosatelites to a recent startup. Basically the young startup will not be able to produce the amount, neither deliver on time. 
#Therefore, it might be interesting to locate the bar/restaurant near to "medium size startups", if there are enough around.

# small_compsdf = data_comp[data_comp['number_of_employees']<300]


In [21]:
#Querying near companies

def smallcomps(df, rad_max_meters=1000):
    lst=[]
    for i in range(len(df)): 
        near=db.companies_clean.find({'$and':[{
                        "geopoint": {
                            "$near": {
                            "$geometry": df[i],
                            "$maxDistance": rad_max_meters
                            }
                        },{
                        'number_of_employees': data_comp['number_of_employees']<500,
                    }
                    }]})
        data=pd.DataFrame(near)
        lst.append(data.shape[0])
    return lst

data_comp['offices_near']=nearComps(data_comp["geopoint"])

SyntaxError: invalid syntax (<ipython-input-21-e61e717eb96a>, line 15)

In [57]:
#If the number of small comapnies (<50 employees) around is big enough, might be interesting to make them our target with the objecrtive of creating the restaurant for synergies. 

def nearComps(df, rad_max_meters=1000):
    lst=[]
    for i in range(len(df)):
        if data_comp['number_of_employees']<50:
            near=db.companies_clean.find({'$and':[{
                            "geopoint": {
                                "$near": {
                                "$geometry": df[i],
                                "$maxDistance": rad_max_meters
                                }
                            }
                        }]})
            data=pd.DataFrame(near)
            lst.append(data.shape[0])
    return lst

small_compsdf['small_offices']=nearComps(small_compsdf["geopoint"])



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [21]:
# Heatmap
latlng = data_comp[['lat', 'lng']].values

# m = folium.Map(location=[40, -45], zoom_start=4)
# m.add_child(plugins.HeatMap(latlng, radius=20))
# m

In [22]:
#Cluster map

m2 = folium.Map(zoom_start=15)
FastMarkerCluster(latlng, radius = 20000000).add_to(m2)
m2


In [23]:
# from pandas.io.json import json_normalize
# json_normalize(data_comp['geopoint'])

In [30]:
# def findNear(df,radio_max_meters = 2000):
#     location = db.geo_offices.find({'main_office':{
#         '$near':{
#             '$geometry':df,
#             '$maxDistance': radio_max_meters,
#         }
#     }
#     })
#     df1 = pd.DataFrame(location)
#     return len(df1)
# near_offices = findNear(data_comp['main_office'][0])


def find_old(df1, radio_max_meters=2000):
    lista=[]
    for i in range(len(df1)): 
        old=db.offices_ok.find({'$and':[{
                        "geo_point": {
                            "$near": {
                                "$geometry": df1[i],
                                "$maxDistance": radio_max_meters,
                            }
                        }
                    },{
                        'is_young':0
                    }]})
        tempdf=pd.DataFrame(old)
        lista.append(tempdf.shape[0])
    return lista


OperationFailure: error processing query: ns=DBcompanies_cb.geo_officesTree: GEONEAR  field=geo_point maxdist=2000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error: unable to find index for $geoNear query

In [27]:
def findNear(df, radio_max_meters = 20):
    lst =[]
    for i in range(len(df)):
        location = db.data_companies_clean.find({'main_office':{
                                                 '$near': {
                                                 '$geometry': df[i],
                                                 '$maxDistance': radio_max_meters,
                                                 }
                                                }
                                                })
        
        df2 = pd.DataFrame(location)
        lst.append(len(df2))    
    return lst

near_offices = findNear(data_comp['main_office'])
len(near_offices)

OperationFailure: error processing query: ns=DBcompanies_cb.geo_officesTree: GEONEAR  field=geo_point maxdist=20 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error: unable to find index for $geoNear query

In [ ]:
data_comp['Near offices'] = pd.DataFrame(near_offices)

In [ ]:
data_comp.head()

In [ ]:
base_map = folium.Map(zoom_start=15)

In [ ]:
a = list(zip(data_comp['lat'], data_comp['lng']))
for i in a:
    b = folium.Marker(location=[i[0],i[1]]).add_to(base_map) 
    
base_map

In [ ]:
from folium.plugins import FastMarkerCluster
base_map_two = folium.Map(zoom_start=15)

FastMarkerCluster(data=list(zip(data_comp['lat'].values, data_comp['lng'].values))).add_to(base_map_two)
base_map_two



In [ ]:
import json
import requests
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
if not "KEYG" in os.environ:
    raise ValueError("You should pass a Google Key")
    
google_key = os.environ["KEYG"]

In [ ]:
BASE_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch'
res = requests.get("{}/json?location=34.017606,-118.487267&radius=1500&type=school&key={}".format(BASE_URL, google_key))





In [ ]:
locat = []
for i in a:
    locat.append(i)



In [ ]:
asd = []
for i in locat:
    asd.append(requests.get("{}/json?location={},{}&radius=1500&type=school&key={}".format(BASE_URL, i[0], i[1], google_key)))
                 


In [ ]:
asd_jsons = []
for i in asd:
    asd_jsons.append(i.json())


In [ ]:
#https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

In [ ]:
BASE_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='


In [ ]:
res = requests.get('{}-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key={}'.format(BASE_URL, google_key)


In [ ]:
BASE_URL = “https://maps.googleapis.com/maps/api/place/nearbysearch”